In [1]:
import ipy_table
from collections import Counter
%run ../aux/postgres.py

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams

import warnings
warnings.filterwarnings('ignore')

tknzr = TweetTokenizer()
#%run functions/preproc.py
#%run functions/classificacao.py

postgres functions loaded...


In [2]:
def tokensCleaner(txt):
    tokens = tknzr.tokenize(txt)
    stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
    clean_tokens = []
    for token in tokens:
        if token[:1] == '#':
            token = token[1:]
        if not token.isalpha():
            continue
        if token in stop:
            continue
        if token[:4] == 'http':
            continue
        if token[:1] == '@':
            continue
        if len(token) <= 3:
            continue

        clean_tokens.append(token)
    return clean_tokens

In [3]:
cmd = ("select cl.codclassificacao, cl.descricao, count(codtweet) "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "where codresultadoanaliseprojeto = 19 "
       "GROUP BY 1 , 2"
      )
res = query(cmd)
res

[(227, 'personal', 34126),
 (226, 'service', 151033),
 (228, 'sale', 8613),
 (229, 'clinical', 59598),
 (223, 'information', 18103),
 (224, 'product', 21833)]

In [4]:
for r in res:
    print(r)
    print(r[1])
    # loading from database
    cmd = ("select t.text, cl.codclassificacao "
           "from resultadoanaliseprojetotweet r "
           "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
           "inner join tweet t on r.codtweet = t.codtweet "
           "where codresultadoanaliseprojeto = 19 and cl.codclassificacao = " + str(r[0]))
    res01 = query(cmd)    
    print(cmd)
    print(len(res01))
    
    # concatenando texto
    txt = ''
    for row in res01:
        txt = txt + ' ' + row[0].lower()    
    
    # pré processamento
    clean_tokens = tokensCleaner(txt)
    
    # unigramas
    unigramas = ngrams(clean_tokens,1)
    unigramas_count = Counter(unigramas).most_common()
    arquivo = "data/unigramas_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "w") as f:
        for b in unigramas_count:
            f.write(b[0][0]+";"+str(b[1])+"\n")    
    
    # bigramas
    bigramas = ngrams(clean_tokens, 2)
    bigramas_count = Counter(bigramas).most_common()
    arquivo = "data/bigramas_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "w") as f:
        for b in bigramas_count:
            f.write(b[0][0]+";"+b[0][1]+";"+str(b[1])+"\n")    

    # trigramas
    trigramas = ngrams(clean_tokens, 3)
    trigramas_count = Counter(trigramas).most_common()
    arquivo = "data/trigramas_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "w") as f:
        for b in trigramas_count:
            f.write(b[0][0]+";"+b[0][1]+";"+b[0][2]+";"+str(b[1])+"\n")    
        
    print()
    

(227, 'personal', 34126)
personal
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 227
34126
data/unigramas_personal.csv
data/bigramas_personal.csv
data/trigramas_personal.csv

(226, 'service', 151033)
service
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 226
151033
data/unigramas_service.csv
data/bigramas_service.csv
data/trigramas_service.csv

(228, 'sale', 8613)
sale
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanalisepr

In [5]:
clean_tokens = tokensCleaner(txt)
print(clean_tokens[:10])

['system', 'surgical', 'brushless', 'motor', 'contra', 'angle', 'handpiece', 'usps', 'torque', 'wrench']


In [6]:
clean_tokens = tokensCleaner(txt)

In [7]:
# unigramas
ungs = ngrams(clean_tokens,1)
aaa = Counter(ungs).most_common()
with open("data/unigramas.csv", "w") as f:
    for b in aaa:
        print("%s;%s" % (b[0][0], b[1] ))
        f.write(b[0][0]+";"+str(b[1])+"\n")

cost;4136
bone;1841
torque;1540
wrench;1509
check;1460
know;1307
straumann;1170
business;985
announce;974
adin;973
chicago;966
social;958
newyork;956
reuters;956
alsgd;956
technology;955
nytimes;954
jimmyfallon;954
ihub;954
bloomberg;954
digitalmarkiting;954
forbes;954
networking;954
foxnews;954
nasdaq;954
thestreet;954
spineguard;936
success;933
system;917
titanium;909
nobel;908
handpiece;862
restoration;798
teeth;798
reduction;790
rate;776
biocare;733
angle;701
contra;675
motor;659
surgical;635
drivers;632
graft;576
placement;572
tooth;567
systems;555
dentalimplants;526
short;523
long;512
screw;505
made;488
study;462
risk;447
types;416
dentistry;412
history;407
successful;394
ceramic;392
broken;389
internal;387
trust;381
need;381
mouth;371
failure;369
removal;367
help;366
surgery;351
remover;349
reservoir;342
people;341
retrieval;341
abutment;338
universal;325
reduces;322
brushless;320
full;314
overall;313
used;313
brief;312
driver;305
approved;303
designed;298
antibacterial;295
invi

answers;7
third;7
link;7
move;7
highly;7
details;7
final;7
durability;7
candidate;7
current;7
competition;7
park;7
supply;7
never;7
sept;7
involve;7
issue;7
clearchoice;7
shown;7
stand;7
touareg;7
feedly;7
vancouver;7
color;7
predictable;7
several;7
says;7
needless;7
newsfile;7
hydraulic;7
multilevel;7
electron;7
determine;7
firms;6
thommen;6
service;6
immediacy;6
submental;6
promo;6
liked;6
thinking;6
reynosa;6
payperclick;6
easy;6
comprising;6
toothreplacement;6
community;6
speedy;6
heart;6
proto;6
take;6
aspects;6
retweet;6
surprise;6
response;6
keeping;6
remains;6
removable;6
alveolar;6
upon;6
atlantaga;6
experienced;6
celebrity;6
organizer;6
regeneration;6
periodontic;6
back;6
pieces;6
influencer;6
tell;6
roughened;6
painless;6
models;6
gradually;6
licensing;6
delivery;6
scan;6
based;6
bingads;6
serf;6
displacement;6
semi;6
ceramics;6
wire;6
cannot;6
preventing;6
inverta;6
tyne;6
symposium;6
factors;6
garlandtx;6
kompressors;6
paper;6
brush;6
sites;6
east;6
wilmslow;6
sides;6
maxi

aluminium;3
placements;3
ahead;3
dentalimplantmonth;3
feature;3
particularly;3
machined;3
multicenter;3
numbers;3
increases;3
louisville;3
regarding;3
manchester;3
depends;3
florida;3
guarantee;3
file;3
bonegrafts;3
associates;3
introduced;3
elements;3
conditions;3
updates;3
discusses;3
jointhetribe;3
dangerous;3
path;3
recap;3
hybrid;3
discovering;3
clinician;3
wireless;3
vimeo;3
degree;3
teaching;3
stents;3
institute;3
gotten;3
presence;3
convenience;3
temperature;3
murray;3
dentalimplantology;3
resulting;3
ions;3
releasing;3
reading;3
vadodara;3
dentistinahmedabad;3
underway;3
cyst;3
impressive;3
improvement;3
wednesdaywisdom;3
newmexico;3
concepts;3
interactive;3
quazgaa;3
stuff;3
cards;3
dentalinstruments;3
screwer;3
previously;3
literally;3
michael;3
closely;3
favorable;3
glass;3
ligament;3
volume;3
friendly;3
peroxide;3
boob;3
buffalo;3
startup;3
damaged;3
array;3
young;3
fitness;3
mtpol;3
contour;3
osteomyelitis;3
yesterday;3
bioactive;3
mondaymotivation;3
gauge;3
followup;3
bl

tested;2
provided;2
dentaltourism;2
heartburn;2
depending;2
googlealerts;2
wight;2
digitaldentistry;2
busting;2
determining;2
obsessive;2
oregon;2
paragon;2
unidentified;2
histomorphometry;2
stripped;2
malvern;2
charge;2
combat;2
gumdisease;2
merged;2
combines;2
surgerycontra;2
deformity;2
odonto;2
bids;2
memorial;2
brushes;2
directly;2
cuomo;2
inserting;2
thin;2
collector;2
risky;2
clears;2
buffalony;2
krishnaraj;2
entered;2
newest;2
woodbridgeva;2
difiaba;2
food;2
late;2
operations;2
chattanooga;2
fits;2
irreversible;2
dentalimplantawarenessmonth;2
neodent;2
cabot;2
employees;2
wallpaper;2
cleft;2
memphis;2
asking;2
defects;2
velocity;2
stabilize;2
attaches;2
confirmed;2
shell;2
forever;2
dearborn;2
rise;2
kinds;2
functional;2
letting;2
michigan;2
closeup;2
cosmetology;2
scaffold;2
suggest;2
segment;2
bioprinting;2
pmma;2
alon;2
puts;2
doha;2
mayan;2
episode;2
opening;2
occurrence;2
emergence;2
women;2
mineolany;2
justifies;2
sauron;2
happiness;2
breakthroughs;2
newslog;2
wondered;2


massage;1
immunohistochemistry;1
starts;1
guest;1
sanitizer;1
promoting;1
agencies;1
dwellers;1
dentalnew;1
anchoring;1
scholaralerts;1
mouths;1
goodman;1
mask;1
obtained;1
forward;1
amnt;1
southernimplantseducation;1
vandyck;1
furthermore;1
recreating;1
triangulation;1
wired;1
cerezen;1
rocket;1
mead;1
philippine;1
madd;1
fourthofjuly;1
teas;1
exploring;1
flooded;1
milwaukee;1
pitney;1
dentalcrown;1
pugetsoundoralsurgery;1
bilingual;1
movement;1
warm;1
knock;1
projects;1
fridayfeeling;1
panic;1
edent;1
evodental;1
dumpster;1
cyprus;1
appreciation;1
cancellation;1
minneapolis;1
thrusdaymotivation;1
grilled;1
barrier;1
sockets;1
famework;1
title;1
proced;1
remarkably;1
lists;1
coolsculpting;1
qualityoflife;1
advertise;1
glasscomposites;1
drive;1
charles;1
civic;1
calculator;1
meters;1
resportion;1
lawyer;1
compressive;1
adapted;1
signed;1
develo;1
newsscientists;1
vena;1
hansen;1
generous;1
nanodiamonds;1
benefitsdentalimplants;1
otherrequired;1
transmitter;1
mounted;1
relations;1
suffe

talents;1
turbine;1
kolkata;1
classy;1
paltop;1
repeated;1
challenge;1
waygoose;1
ducts;1
additions;1
exporter;1
sadc;1
implantsadvantages;1
chemicals;1
equinox;1
malo;1
entertain;1
ricky;1
complexity;1
offerings;1
beverlyma;1
drummond;1
palette;1
irresistible;1
uniqueness;1
dates;1
zerconia;1
rice;1
sunnyvaleca;1
healed;1
regrowing;1
healthbeat;1
valspar;1
toxic;1
robert;1
additi;1
celebrities;1
pens;1
spelling;1
gamechanger;1
madhura;1
vlog;1
yttria;1
specialised;1
ncbi;1
ebifpocs;1
gems;1
dentalcarematters;1
immediatedentalimplant;1
jade;1
married;1
minor;1
ontheblog;1
bearing;1
implantations;1
stratasys;1
heel;1
saintlouispark;1
staffing;1
kansasperio;1
lakeridgedentalcare;1
layer;1
インプラント;1
competent;1
jacks;1
webb;1
boffinjournals;1
molloy;1
cosmetics;1
marketwired;1
ibraheem;1
dentistchennai;1
peroperative;1
tells;1
dentistinlondon;1
twinning;1
jacobs;1
recarga;1
folding;1
aeton;1
testimonial;1
dentalimplantsrock;1
shifting;1
clearly;1
remember;1
canyon;1
rabbits;1
bowhousedenta

bankruptcy;1
sertraline;1
identify;1
basalimplant;1
knocked;1
marina;1
eliminate;1
disadvantage;1
branchburgoralsurgery;1
lowest;1
fixing;1
rake;1
synthesized;1
dominant;1
austintx;1
granddaughters;1
aofsdentalimplants;1
thierry;1
trustthe;1
salts;1
incissor;1
prominence;1
bags;1
knew;1
israelius;1
songs;1
aspired;1
surety;1
oralimplantology;1
facilitates;1
cabinet;1
medicalnews;1
denti;1
students;1
thewoodlandstx;1
sauhy;1
jurnal;1
endovascular;1
orasana;1
econo;1
significance;1
traced;1
approval;1
arlington;1
microscope;1
chelseany;1
photographs;1
madris;1
enquiries;1
dentalheal;1
battery;1
sina;1
napa;1
schizophrenia;1
majors;1
bathrooms;1
broadly;1
electric;1
laptop;1
tiptuesday;1
amounts;1
enviro;1
blackheath;1
king;1
prosthodontist;1
socialdentistry;1
prepared;1
artex;1
immunological;1
hadi;1
surgicalphotos;1
dentaltourismexico;1
lutherville;1
questionable;1
exton;1
medicaltourism;1
trend;1
replicas;1
looming;1
safest;1
gram;1
shareholders;1
gadental;1
compli;1
proliferation;1
al

In [8]:
# bigramas
ungs = ngrams(clean_tokens, 2)
aaa = Counter(ungs).most_common()
for b in aaa:
    print("%s;%s;%s" % (b[0][0], b[0][1], b[1] ))

cost;cost;2804
torque;wrench;1328
social;networking;954
business;foxnews;954
ihub;newyork;954
networking;business;954
digitalmarkiting;ihub;954
forbes;nasdaq;954
jimmyfallon;forbes;954
reuters;bloomberg;954
newyork;social;954
nasdaq;chicago;954
nytimes;reuters;954
chicago;digitalmarkiting;954
bloomberg;thestreet;954
thestreet;jimmyfallon;954
spineguard;adin;933
foxnews;alsgd;826
nobel;biocare;728
success;rate;702
contra;angle;662
bone;graft;523
check;torque;505
placement;nytimes;495
alsgd;announce;495
adin;systems;475
systems;announce;468
alsgd;spineguard;461
announce;spineguard;461
adin;nytimes;459
technology;placement;456
announce;technology;454
broken;screw;378
angle;handpiece;371
technology;trust;368
trust;technology;364
short;long;351
screw;removal;330
reduction;contra;323
removal;remover;307
retrieval;restoration;307
remover;retrieval;307
overall;success;305
reservoir;reduces;298
brief;history;279
press;ceramic;252
hydrualic;press;251
check;straumann;249
australian;approved;247
a

know;want;12
preferred;corp;12
infection;bone;12
news;alert;12
abutments;video;12
handpiece;torque;12
goldadapt;engaging;12
tooth;natural;12
system;procedures;12
dentistry;invisalign;12
provide;bone;12
handpiece;cordless;12
biocare;goldadapt;12
harvard;health;12
preserve;natural;12
health;barthmanndentureclinic;12
titanium;zirconium;12
might;provide;12
sale;motor;12
years;become;12
surgery;brushless;12
healing;abutment;12
biomechanical;components;12
invisalign;charlottedentistry;12
failure;dentistry;12
fight;infections;12
replacement;cosmeticdentist;12
video;news;12
improved;technology;12
grafting;material;12
allow;teething;12
cellulose;might;12
short;check;12
lamina;internal;12
sterilized;spiral;12
sirona;lead;12
dentalimplants;falseteeth;12
oral;health;12
plant;cellulose;12
graft;support;12
alternative;methods;12
wear;cost;12
replace;abutment;12
proven;preserve;12
support;prosthesis;12
handpiece;control;12
bone;mill;12
guide;check;12
jawbone;need;12
become;desirable;12
teething;troub

connection;active;8
british;columbia;8
require;bone;8
nobelreplace;tapered;8
conical;nobel;8
mount;instruments;8
latest;article;8
combo;check;8
cost;motor;8
connection;regular;8
usps;system;8
titanium;bone;8
straumann;published;8
percent;help;8
check;neobiotech;8
crown;marylebone;8
operation;mini;8
smileline;screwdriver;8
internal;brand;8
help;choose;8
tosi;instrument;8
know;three;8
cost;facts;8
interesting;facts;8
design;surface;8
didyouknow;made;8
advanced;tooth;8
look;feel;8
gene;combination;8
enough;bone;8
mini;drivers;8
bone;cost;8
wrench;knob;8
abutments;shoulder;8
drivers;star;8
machine;reduction;8
know;implantsprocenter;8
worldwide;generate;8
handpiece;ship;8
preset;premium;8
cost;mini;8
explains;fence;8
clip;transfer;8
straumann;tools;8
titanium;rockets;8
platform;surgery;8
type;motors;8
factor;success;8
instruments;sealed;8
insert;transfer;8
ever;developed;8
zimmer;straumann;8
getting;need;8
replacement;tooth;8
cost;people;8
bridge;caries;8
scaling;worldwide;8
history;planned

arrowdentalcentre;cost;5
proprietary;fulcrum;5
sufficient;bone;5
closest;healthy;5
failure;medical;5
dentist;torque;5
atrophic;posterior;5
teeth;vale;5
medical;handpiece;5
create;strong;5
unveil;suite;5
swiss;firms;5
process;getting;5
graft;westseneca;5
help;visit;5
know;every;5
spray;handpiece;5
rate;read;5
wrench;antibacterial;5
europe;system;5
complete;guide;5
successful;therapy;5
stopper;system;5
straumann;great;5
platform;alpha;5
facts;help;5
acclaimed;surface;5
three;torques;5
reduce;infection;5
cost;learn;5
traditional;typically;5
maker;straumann;5
information;maui;5
driver;surgical;5
slow;contra;5
importance;stability;5
wrench;hiossen;5
adapter;inner;5
failure;rates;5
really;replace;5
alphabio;surgery;5
study;numerical;5
proven;success;5
suite;northeast;5
instrument;sale;5
project;scaling;5
years;dentalimplants;5
dentistry;titanium;5
sells;stake;5
aaid;american;5
ceramic;titanium;5
abutment;titanium;5
distributors;proprietary;5
express;post;5
phar;researchers;5
know;taking;5
in

opened;straumann;3
wonders;beauty;3
surgical;coxo;3
system;unit;3
difficult;questions;3
beauty;style;3
amazing;scientific;3
orthopedic;application;3
read;system;3
know;torque;3
range;bone;3
instead;choose;3
short;nobel;3
waiting;dentalimplants;3
dentist;drjosephmoussa;3
heights;process;3
review;influence;3
type;metal;3
dentistry;contra;3
slide;complications;3
service;artificial;3
dentistry;million;3
according;needs;3
replacement;missing;3
restoration;bridge;3
nail;noble;3
printing;already;3
history;stem;3
matters;order;3
project;mabb;3
process;cost;3
mouth;mini;3
case;report;3
placement;torque;3
procedures;cost;3
facial;aesthetics;3
posts;system;3
improving;stability;3
metal;free;3
rein;torque;3
material;powerpoint;3
healthcare;dentis;3
solid;permanent;3
people;used;3
needs;done;3
exclusive;licensing;3
coupling;nobel;3
narrow;solid;3
lansingmi;stjohnsmi;3
india;nanocoating;3
style;love;3
titanium;printing;3
cosmeticdentistry;shermanoaks;3
rescue;repair;3
ceramic;quick;3
similar;items;3

...

shows;high;1
name;please;1
foundation;cost;1
research;systematicreview;1
post;make;1
group;dentalnews;1
healing;calvarial;1
sino;coastal;1
look;real;1
steel;cost;1
alloy;fact;1
valve;structures;1
prosthesis;advantages;1
universal;drills;1
made;determines;1
supercars;create;1
invalid;based;1
digitaldentistry;modern;1
successful;invisalign;1
found;teeth;1
rita;ranch;1
generation;affordability;1
apparently;osteotome;1
deal;pros;1
replacement;technique;1
check;success;1
effects;company;1
mini;advanceddental;1
split;receive;1
protocol;comparative;1
around;benefits;1
megagen;demonstration;1
assisted;system;1
titanium;learning;1
teeth;post;1
wisconsin;kenosha;1
designed;solitary;1
every;dentist;1
tooth;arram;1
zirconia;alternative;1
treatments;technology;1
style;type;1
elkgrovedentalimplants;sacramentodentist;1
cost;successful;1
blog;complete;1
evaluation;interface;1
done;please;1
sale;titanium;1
survival;reddit;1
sino;machine;1
review;ball;1
girls;american;1
grafting;wisdom;1
fullscope;order

pack;style;1
dentistry;synthetic;1
broker;certification;1
instrument;amazing;1
total;facial;1
glamorgan;choose;1
dentures;designed;1
support;fixed;1
beautifulsmile;phoenix;1
impla;full;1
life;multiple;1
angle;scientists;1
wrench;head;1
higher;risk;1
risk;bone;1
risk;successful;1
permadontics;heard;1
nano;akhund;1
impact;costs;1
americans;impact;1
costs;check;1
partha;dentalimplants;1
group;made;1
clinical;dentistry;1
material;slim;1
dentalhealth;american;1
flaps;know;1
denture;comfort;1
aesthetic;facts;1
aimed;investigate;1
learn;winnipeg;1
dentistry;refer;1
goes;straight;1
study;million;1
drainage;made;1
fabrication;patented;1
post;insertion;1
years;miss;1
インプラント;審美歯科;1
cause;serious;1
show;several;1
like;alphabio;1
edent;bioconcept;1
dentistry;developed;1
procedures;training;1
device;manufacturing;1
cost;failure;1
dentalimplants;making;1
instrument;lmws;1
recent;example;1
blog;simple;1
making;frontier;1
piece;four;1
technology;conveyor;1
recovery;long;1
healthy;please;1
attached;scre

protocol;immediate;1
fixed;straumann;1
minidentalimplants;bocaraton;1
drugs;torque;1
drummond;avery;1
rate;techniques;1
rodstewart;artificial;1
diagnosis;predictable;1
abutments;branded;1
hand;doctor;1
know;products;1
simple;broadly;1
learned;drink;1
type;reduce;1
follow;boland;1
harrow;severe;1
methods;bone;1
made;rental;1
experimentation;resulted;1
line;shopping;1
brower;introduction;1
cost;avenue;1
surgeons;worldwide;1
dist;torque;1
removal;spiral;1
smart;training;1
help;consist;1
care;percent;1
teeth;brentford;1
titanium;credit;1
plastic;made;1
cadaver;study;1
types;need;1
future;phuket;1
million;north;1
role;facts;1
smile;artificial;1
biohorizons;uses;1
cost;khabar;1
combo;straumann;1
achievement;primary;1
hygiene;last;1
sleeve;approach;1
biological;performance;1
system;improve;1
upset;internal;1
districts;berkeley;1
advanc;cost;1
service;orange;1
people;could;1
mini;single;1
shorter;recovery;1
straumann;gicprivatelimited;1
michigan;titanium;1
reservoir;helps;1
bodily;fluids;1
fou

expansion;bone;1
graft;everything;1
replace;root;1
even;patients;1
fear;surgical;1
teeth;around;1
powerfull;cost;1
zimmer;ceramic;1
investment;dentalimplants;1
zirconia;hybrid;1
watercolor;print;1
healthy;functioning;1
palette;spend;1
influence;primary;1
brand;drivers;1
student;ratio;1
evaluation;artifacts;1
biofilms;patients;1
phosphate;coat;1
parametric;study;1
made;keys;1
validity;thier;1
placing;dentalimplants;1
allergy;related;1
seattle;antibacterial;1
sensublog;surprisingly;1
surgical;next;1
advertisements;really;1
patient;know;1
failure;compli;1
rate;improved;1
cost;transfer;1
failure;rental;1
melting;titanium;1
ikea;machine;1
jenny;craig;1
repeatedly;sees;1
glenn;mcevoy;1
rescue;ship;1
know;really;1
received;announce;1
everything;treatment;1
osteoporotic;fracture;1
surgical;alsgd;1
brushing;minutes;1
cricket;lfnin;1
discussed;conical;1
last;average;1
dentalimplants;need;1
bridge;cutting;1
throat;link;1
right;decision;1
hold;tooth;1
establish;foundation;1
metal;made;1
lingo;peop

prosthesis;noticeably;1
green;material;1
center;serendipity;1
made;health;1
talk;titanium;1
sirona;different;1
treatment;option;1
injections;different;1
stefan;bone;1
chart;meat;1
places;mouth;1
learn;visit;1
better;success;1
beverlyhills;elmonte;1
found;bone;1
rotten;tooth;1
month;little;1
machination;applications;1
expanders;titanium;1
paul;korb;1
mead;bone;1
magnetic;resonance;1
weight;fitn;1
brownstown;tooth;1
considering;undergo;1
problem;therapy;1
cost;effect;1
risks;simple;1
sealed;system;1
terms;strength;1
right;side;1
come;types;1
lutherville;facial;1
kill;subject;1
startup;medtech;1
symposium;straumann;1
aids;studies;1
saghatchi;blog;1
patients;systemic;1
bone;naturally;1
integrati;artificial;1
times;procedures;1
sponsor;already;1
solution;discover;1
harvested;proximal;1
antoun;hadi;1
introducing;ceramic;1
precisely;weld;1
damn;congrats;1
proprietary;full;1
profitability;professionalism;1
used;tooth;1
periodontal;associates;1
system;whale;1
wrench;handle;1
alpha;zimmer;1
sale

In [9]:
# trigramas
ungs = ngrams(clean_tokens, 3)
aaa = Counter(ungs).most_common()
for b in aaa:
    print("%s;%s;%s;%s" % (b[0][0], b[0][1], b[0][2], b[1]))

cost;cost;cost;2137
jimmyfallon;forbes;nasdaq;954
social;networking;business;954
forbes;nasdaq;chicago;954
ihub;newyork;social;954
chicago;digitalmarkiting;ihub;954
newyork;social;networking;954
reuters;bloomberg;thestreet;954
nasdaq;chicago;digitalmarkiting;954
bloomberg;thestreet;jimmyfallon;954
digitalmarkiting;ihub;newyork;954
nytimes;reuters;bloomberg;954
networking;business;foxnews;954
thestreet;jimmyfallon;forbes;954
business;foxnews;alsgd;826
placement;nytimes;reuters;495
check;torque;wrench;473
spineguard;adin;systems;469
adin;systems;announce;468
announce;spineguard;adin;461
alsgd;spineguard;adin;461
systems;announce;spineguard;461
spineguard;adin;nytimes;459
adin;nytimes;reuters;459
technology;placement;nytimes;454
alsgd;announce;technology;454
announce;technology;placement;454
foxnews;alsgd;spineguard;415
foxnews;alsgd;announce;411
trust;technology;trust;364
contra;angle;handpiece;364
technology;trust;technology;364
broken;screw;removal;327
reduction;contra;angle;315
remove

astra;dentsply;biohorizons;10
abutment;narrow;platform;10
image;solidedge;maker;10
events;product;videos;10
missing;teeth;know;10
mini;theyre;affordable;10
slides;powerpoint;slide;10
behind;technology;various;10
commercially;pure;titanium;10
long;straumann;torque;10
develop;helps;prevent;10
call;office;work;10
straumann;long;surgery;10
post;blog;study;10
highest;success;rate;10
square;precision;precise;10
nobrpl;nobel;biocare;10
success;rate;cost;10
find;hiossen;products;10
teething;troubles;allow;10
technology;various;types;10
know;cost;cost;10
made;hint;strong;10
nobel;biocare;check;10
adapter;many;different;10
cost;cost;alsgd;10
call;facts;success;10
solidedge;maker;biotec;10
leuven;researchers;develop;10
handpiece;machine;motor;10
contra;angle;coxo;10
odds;read;check;10
destroyingclip;hydrualic;press;10
wrench;straumann;torque;10
retweeted;daily;life;10
expander;bone;graft;10
maui;worth;visiting;10
heard;mini;theyre;10
system;spiral;type;10
drill;brushless;motor;10
shout;brain;alie

biocompatible;made;titanium;7
brand;spiral;angulated;7
type;reduction;motor;7
news;study;identifies;7
roots;made;titanium;7
failures;multilevel;mixed;7
weird;history;cost;7
handpiece;kavo;ebaydailybids;7
used;explores;different;7
restoration;minimally;invasive;7
contra;angle;root;7
surgical;push;style;7
drivers;hexa;type;7
platform;sterilized;spiral;7
graft;totalimplant;check;7
brand;straumann;short;7
nobel;biocare;square;7
integration;bone;health;7
torque;drivers;long;7
bone;synthetic;materials;7
bone;mass;adding;7
rate;success;affected;7
fabricate;ideal;temporary;7
coxo;surgery;brushless;7
handpiece;endo;contra;7
synthetic;materials;used;7
implantology;zirconia;titanium;7
scientists;developed;containing;7
platform;internal;transfer;7
free;shipping;abutment;7
artificial;tooth;root;7
handpiece;antibacterial;activity;7
titanium;dentist;straight;7
drugs;affect;integration;7
retrieval;restoration;next;7
story;scary;paul;7
surgical;contra;angle;7
cost;cost;sponsored;7
reduce;failure;onmedi

made;metal;helps;4
time;make;last;4
achieve;success;immediate;4
canal;mete;system;4
environment;placing;bone;4
tack;bone;graft;4
closed;tray;internal;4
teeth;mouth;amazing;4
surface;functionalization;titanium;4
thinking;getting;need;4
straight;abutment;conical;4
straight;abutment;normal;4
impact;different;surface;4
twopiece;zirconia;twopiece;4
crusher;titanium;plates;4
check;tack;graft;4
nobelbiocare;torque;wrench;4
slow;motor;surgical;4
conta;angle;cost;4
reliable;straumann;system;4
button;style;reduction;4
need;know;bone;4
bone;torque;wrench;4
answer;surprise;cost;4
charlottedentist;beautifulsmile;cost;4
types;powerpoint;cost;4
wrench;million;americans;4
cost;learning;cost;4
narrow;abutments;internal;4
missing;teeth;keep;4
long;time;make;4
universal;knob;check;4
appointment;know;booking;4
reduce;infection;risk;4
titanium;plates;bone;4
metal;allergy;check;4
astm;quality;shaanxi;4
bone;morselizer;bone;4
dentalimplants;cosmeticdentistry;garlandtx;4
successful;lfnin;nanocoating;4
level;n

removal;exposed;titanium;3
crestal;bone;loss;3
bayarea;dentalhealth;losaltos;3
morphological;evolution;tissue;3
risk;failure;irradiated;3
reynosa;reddit;posted;3
thommen;nobel;biocare;3
dentist;next;future;3
angle;handpiece;know;3
know;someone;probably;3
tissue;interactions;ceramic;3
reduction;handpiece;style;3
typically;used;situations;3
learn;common;problems;3
wrong;cost;cost;3
biocare;screwdriver;cost;3
cost;hydrualic;press;3
peek;torque;wrench;3
today;made;titanium;3
push;cicada;reduction;3
driver;straumann;torque;3
bone;graft;necessary;3
teeth;feel;function;3
instead;traditional;broken;3
corrosion;instead;choose;3
people;need;pricing;3
meaning;titanium;alloy;3
patient;read;story;3
alloy;titanium;different;3
transmission;electron;microscopic;3
percent;know;overall;3
photobiomodulation;therapy;improving;3
whether;periods;validity;3
canadian;developer;innovative;3
located;average;success;3
replace;abutment;nobel;3
reduction;handpiece;slow;3
endo;motor;system;3
surgery;motor;complete;

...

hexagon;system;straumann;1
insertion;bilateral;traumatic;1
right;mouth;brief;1
journal;prosthodontics;wiley;1
polymers;abutments;torque;1
dentistry;proud;announce;1
history;progression;present;1
bone;graft;smokers;1
cancetlous;bone;clinician;1
cost;cost;components;1
titanium;made;keys;1
titanium;titanium;dentalimplants;1
facts;know;cost;1
four;done;quality;1
narrow;location;info;1
quality;assessment;site;1
ceramic;presents;iaomt;1
success;ambitious;estimates;1
tube;surgical;endodontics;1
titanium;used;benefits;1
permadontics;hydrualic;press;1
warrenoh;know;learn;1
materials;root;canal;1
types;types;mini;1
ceramics;composites;sintering;1
growth;brushless;surgical;1
post;types;hawaiifamilydental;1
dentalimplants;willowpassdentalcare;state;1
company;announ;narrow;1
surgical;million;americans;1
peroxide;study;evaluate;1
surgeon;full;story;1
masterclass;dentalimplants;straumann;1
print;design;vector;1
states;american;academy;1
best;market;doctor;1
rosxzzge;study;identifies;1
traditional;typ

unlike;dentures;biohorizons;1
model;impression;template;1
mdis;dentalimplants;smaller;1
cost;artificial;tooth;1
surgical;difiaba;hair;1
implantpathway;posted;international;1
replaced;cool;research;1
degrees;internal;bone;1
therapy;qualities;largo;1
indilens;hollywood;nanocoating;1
know;full;tyne;1
surface;picdental;impression;1
opportunity;territory;sales;1
subperiosteal;bone;perfect;1
replacement;find;made;1
complications;learn;replacement;1
engineering;improved;techniques;1
brands;seconds;cost;1
brand;teeth;government;1
replacement;teeth;cases;1
charlottedentist;beautifulsmile;percent;1
printing;fabrication;zirconia;1
hair;barbecue;sauce;1
flow;sensor;surgery;1
yoga;cost;everything;1
hole;check;drivers;1
biocompatible;grow;teeth;1
accommodated;basal;cortical;1
immediate;really;worth;1
celgro;study;accelerating;1
know;costs;make;1
high;performance;superior;1
treatment;surgical;component;1
teeth;mouth;perfect;1
expander;trimmer;conical;1
machine;could;metal;1
osteotomy;preparation;cost

advert;radio;company;1
zimmer;powerpoint;presentations;1
bone;graft;term;1
zirconia;ceraroot;conservative;1
cost;cost;irradiated;1
info;visit;broken;1
adin;systems;global;1
advantages;million;people;1
right;left;screws;1
mouth;using;coxo;1
decide;hand;replace;1
cost;sinus;lifting;1
years;simple;gasser;1
cosmeticdentist;seeing;oral;1
comment;howardfarran;researchers;1
imaging;style;reduction;1
handpiece;gift;pros;1
bone;maxon;motor;1
free;shipping;socket;1
pros;cons;style;1
material;foll;regular;1
closely;associated;successful;1
pros;history;cost;1
slideshare;people;replacing;1
could;anything;like;1
allograft;bone;grafting;1
aspect;overview;corrosion;1
restoration;help;studio;1
bone;graft;mini;1
barthmanndentureclinic;denturist;know;1
know;bravo;motor;1
system;technologies;first;1
aesthetics;without;appearing;1
endo;motor;narrow;1
thommen;torque;wrench;1
treatment;including;bone;1
tool;replace;tooth;1
showcases;products;services;1
torque;wrench;manufacturer;1
helpful;know;materials;1
fa

closely;imitate;lost;1
augmentation;machine;drill;1
read;make;year;1
pans;sets;bone;1
sale;replace;surgical;1
solutions;machine;surgical;1
around;keep;tooth;1
improved;upon;today;1
canal;cracked;zombie;1
dentalimplants;falseteeth;sanramonca;1
successful;health;care;1
dentist;complete;could;1
infections;myhillsdentist;antibacterial;1
forteimplantcenter;know;overall;1
surgicalphotos;long;last;1
system;totalimplantcom;check;1
reconstruction;according;recent;1
maui;businesses;offering;1
yttria;stabilized;zirconia;1
height;inserting;shorter;1
nobelactive;sealed;bone;1
know;benefits;using;1
therapy;rockets;something;1
makes;tick;whether;1
cost;usually;made;1
collateral;alsgd;spineguard;1
nanocoating;successful;heard;1
process;cost;understanding;1
cell;allstar;auto;1
average;cost;high;1
suppliers;cost;bone;1
health;medicine;innovations;1
surgical;finding;right;1
material;grow;teeth;1
internal;short;wide;1
certification;companies;tucson;1
available;tissue;level;1
cannot;cavities;however;1
init

extraction;used;placing;1
evolution;system;motor;1
location;check;maui;1
rate;angulated;multi;1
handpiece;made;army;1
mondaymood;root;canal;1
check;evolved;provide;1
nobel;biocare;surprising;1
surgical;instruments;check;1
people;info;alsgd;1
push;button;analog;1
thei;check;torque;1
bone;augmentation;grafting;1
help;visit;smile;1
carotid;artery;disease;1
maui;cost;fact;1
contribution;dentistry;around;1
augmentation;rehabilitation;edentulous;1
retrieval;good;explanation;1
bone;grafting;contra;1
multiple;failing;teeth;1
rein;cost;facts;1
iphone;ikea;spiral;1
cost;cost;healing;1
live;prosthesis;dentallab;1
contra;angle;avoid;1
sheet;report;cases;1
types;health;healthy;1
rate;remarkably;higher;1
removing;hydrualic;press;1
business;foxnews;people;1
analyzes;systematic;reviews;1
facts;infographic;medical;1
instrument;tray;made;1
compatible;internal;name;1
abutment;crwon;environmental;1
dentistry;studies;show;1
bacterial;infection;torque;1
solid;approved;antibacterial;1
aesthetics;hisham;mousl

In [10]:
cmd = ("select t.text, cl.codclassificacao "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "inner join tweet t on r.codtweet = t.codtweet "
       "where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 228 ")
res01 = query(cmd)

In [11]:
print(len(res01))

8613


In [12]:
res01[:10]

[('RT @41Strange: Dental Implant Insurance Ad on bowling lanes, Germany https://t.co/uhVKEP8LrM',
  228),
 ('From USA! TDX Dental Implant System Motor +NSK Style 20:1 Push Button Handpiece  https://t.co/jlsh2HeHvj',
  228),
 ('dental implants abutment analog lab \nhttps://t.co/Q6GtxnPkp3', 228),
 ('dental implants abutment analog lab \nhttps://t.co/Q6GtxnPkp3', 228),
 ('30+ year old dental implant?! https://t.co/6XfzgJFArx', 228),
 ('dental implants abutment analog lab \nhttps://t.co/Q6GtxnPkp3', 228),
 ('dental implants abutment analog lab \nhttps://t.co/Q6GtxnPkp3', 228),
 ('dentistry dental implant restoration surgery tools \nhttps://t.co/Q6GtxnPkp3',
  228),
 ('dentistry dental implant restoration surgery tools \nhttps://t.co/Q6GtxnPkp3',
  228),
 ('dental implants abutment analog lab \nhttps://t.co/Q6GtxnPkp3', 228)]

In [13]:
txt = ''
for r in res01:
    txt = txt + ' ' + r[0].lower()

In [14]:
print(txt[:1000])

 rt @41strange: dental implant insurance ad on bowling lanes, germany https://t.co/uhvkep8lrm from usa! tdx dental implant system motor +nsk style 20:1 push button handpiece  https://t.co/jlsh2hehvj dental implants abutment analog lab 
https://t.co/q6gtxnpkp3 dental implants abutment analog lab 
https://t.co/q6gtxnpkp3 30+ year old dental implant?! https://t.co/6xfzgjfarx dental implants abutment analog lab 
https://t.co/q6gtxnpkp3 dental implants abutment analog lab 
https://t.co/q6gtxnpkp3 dentistry dental implant restoration surgery tools 
https://t.co/q6gtxnpkp3 dentistry dental implant restoration surgery tools 
https://t.co/q6gtxnpkp3 dental implants abutment analog lab 
https://t.co/q6gtxnpkp3 dentistry dental implant restoration surgery tools 
https://t.co/q6gtxnpkp3 us stock azdent dental implant system  a-cube(110v)  usps   hot sale https://t.co/vkhs7ryg5v us stock azdent dental implant system  a-cube(110v)  usps   hot sale https://t.co/kx3vbiaizh us stock azdent dental impla

In [15]:
aaaa = 'uga'
print(aaaa[1:])

ga


In [16]:
tokens = tknzr.tokenize(txt)
print(tokens[:10])
stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
clean_tokens = []
for token in tokens:
    if token[:1] == '#':
        token = token[1:]
    if not token.isalpha():
        continue
    if token in stop:
        continue
    if token[:4] == 'http':
        continue
    if token[:1] == '@':
        continue
    if len(token) <= 3:
        continue
        
    clean_tokens.append(token)
print(clean_tokens[:10])

['rt', '@41strange', ':', 'dental', 'implant', 'insurance', 'ad', 'on', 'bowling', 'lanes']
['insurance', 'bowling', 'lanes', 'germany', 'system', 'motor', 'style', 'push', 'button', 'handpiece']


In [17]:
ungs = ngrams(clean_tokens,1)
aaa = Counter(ungs).most_common()
for b in aaa:
    print(b)

(('insurance',), 1518)
(('handpiece',), 1340)
(('cost',), 1052)
(('surgical',), 953)
(('abutment',), 950)
(('angle',), 931)
(('contra',), 921)
(('check',), 915)
(('idea',), 884)
(('surprise',), 882)
(('sponsored',), 867)
(('listings',), 849)
(('surgery',), 821)
(('reduction',), 727)
(('speed',), 608)
(('motor',), 568)
(('save',), 462)
(('screw',), 456)
(('bone',), 443)
(('money',), 435)
(('costa',), 426)
(('system',), 426)
(('rica',), 425)
(('restoration',), 409)
(('straight',), 374)
(('affordable',), 365)
(('analog',), 362)
(('tools',), 351)
(('brushless',), 343)
(('dentistry',), 330)
(('cheap',), 287)
(('free',), 278)
(('broken',), 261)
(('instruments',), 252)
(('external',), 219)
(('push',), 211)
(('crown',), 208)
(('prosthetics',), 193)
(('drill',), 191)
(('internal',), 189)
(('sinus',), 188)
(('removal',), 188)
(('tooth',), 186)
(('nose',), 184)
(('screen',), 184)
(('remover',), 178)
(('lift',), 177)
(('teeth',), 172)
(('germany',), 168)
(('bowling',), 167)
(('lanes',), 167)
(('un

(('osstem',), 10)
(('californiadentist',), 10)
(('open',), 10)
(('vegas',), 10)
(('afford',), 10)
(('virginia',), 10)
(('beauty',), 10)
(('school',), 10)
(('campbelldentalimplants',), 10)
(('last',), 10)
(('prevention',), 10)
(('cars',), 10)
(('denverdentalimplants',), 10)
(('costofimplants',), 10)
(('south',), 10)
(('chicago',), 10)
(('tack',), 9)
(('driver',), 9)
(('northern',), 9)
(('months',), 9)
(('video',), 9)
(('hollow',), 9)
(('life',), 9)
(('combine',), 9)
(('adjusting',), 9)
(('assistant',), 9)
(('dentaires',), 9)
(('article',), 9)
(('short',), 9)
(('even',), 9)
(('cutting',), 9)
(('memphis',), 9)
(('versus',), 9)
(('cases',), 9)
(('discover',), 9)
(('encircles',), 9)
(('hard',), 9)
(('necessary',), 9)
(('sydent',), 9)
(('mallet',), 9)
(('angular',), 9)
(('procedures',), 9)
(('manager',), 9)
(('apply',), 9)
(('status',), 9)
(('sets',), 9)
(('handle',), 9)
(('involved',), 8)
(('adin',), 8)
(('crwon',), 8)
(('made',), 8)
(('mouth',), 8)
(('pearland',), 8)
(('ways',), 8)
(('losi

(('kurtis',), 2)
(('integral',), 2)
(('prosthodontic',), 2)
(('body',), 2)
(('intra',), 2)
(('توظيف',), 2)
(('odontologia',), 2)
(('funds',), 2)
(('whole',), 2)
(('sirona',), 2)
(('provides',), 2)
(('antivirus',), 2)
(('chevy',), 2)
(('vicksburg',), 2)
(('risk',), 2)
(('brands',), 2)
(('damaged',), 2)
(('container',), 2)
(('remedies',), 2)
(('diameter',), 2)
(('georgia',), 2)
(('tuesdaymotivation',), 2)
(('bangladesh',), 2)
(('scraper',), 2)
(('dealers',), 2)
(('supports',), 2)
(('elos',), 2)
(('socket',), 2)
(('disk',), 2)
(('pinnacle',), 2)
(('parsippany',), 2)
(('thirty',), 2)
(('webnetwork',), 2)
(('deductions',), 1)
(('furniture',), 1)
(('medentra',), 1)
(('riyadh',), 1)
(('club',), 1)
(('fiberoptic',), 1)
(('icymi',), 1)
(('load',), 1)
(('interrested',), 1)
(('environmental',), 1)
(('albuquerque',), 1)
(('chryslerse',), 1)
(('apple',), 1)
(('vyvanse',), 1)
(('steamboat',), 1)
(('hartland',), 1)
(('quite',), 1)
(('marketplace',), 1)
(('frills',), 1)
(('directly',), 1)
(('licks',),

(('wins',), 1)
(('answers',), 1)
(('booking',), 1)
(('researchers',), 1)
(('burtonsville',), 1)
(('molds',), 1)
(('sleeping',), 1)
(('sarasota',), 1)
(('dynamic',), 1)
(('woolen',), 1)
(('detachable',), 1)
(('bergen',), 1)
(('ticket',), 1)
(('hyderabad',), 1)
(('fabrication',), 1)
(('sitting',), 1)
(('popcorn',), 1)
(('forward',), 1)
(('bonegrafts',), 1)
(('coupling',), 1)
(('barbecue',), 1)
(('lacrosse',), 1)
(('businesslisting',), 1)
(('londondentalimplants',), 1)
(('move',), 1)
(('architecture',), 1)
(('hygiene',), 1)
(('adverts',), 1)
(('alabama',), 1)
(('skull',), 1)
(('manufactur',), 1)
(('bathroom',), 1)
(('syze',), 1)
(('lexus',), 1)
(('maxfax',), 1)
(('protect',), 1)
(('higher',), 1)
(('newyork',), 1)
(('molar',), 1)
(('inspire',), 1)
(('lakhs',), 1)
(('flushing',), 1)
(('keeps',), 1)
(('assured',), 1)
(('bionika',), 1)
(('recycling',), 1)
(('compliant',), 1)
(('reebok',), 1)
(('cauvery',), 1)
(('leviathan',), 1)
(('pregnant',), 1)
(('wisdomteeth',), 1)
(('mozes',), 1)
(('supp

In [18]:
bgs = ngrams(clean_tokens,2)
aaa = Counter(bgs).most_common()
for b in aaa:
    print(b)

(('contra', 'angle'), 917)
(('cost', 'surprise'), 879)
(('idea', 'cost'), 849)
(('surprise', 'sponsored'), 849)
(('sponsored', 'listings'), 849)
(('listings', 'idea'), 579)
(('angle', 'handpiece'), 528)
(('costa', 'rica'), 425)
(('save', 'money'), 425)
(('money', 'costa'), 414)
(('rica', 'save'), 407)
(('speed', 'contra'), 383)
(('reduction', 'speed'), 378)
(('surgery', 'tools'), 313)
(('abutment', 'analog'), 307)
(('restoration', 'surgery'), 305)
(('dentistry', 'restoration'), 305)
(('surgical', 'brushless'), 226)
(('broken', 'screw'), 217)
(('brushless', 'motor'), 205)
(('reduction', 'contra'), 204)
(('check', 'bone'), 186)
(('sinus', 'lift'), 175)
(('insurance', 'affordable'), 174)
(('surgical', 'straight'), 174)
(('straight', 'handpiece'), 173)
(('handpiece', 'nose'), 171)
(('insurance', 'bowling'), 167)
(('bowling', 'lanes'), 167)
(('lanes', 'germany'), 167)
(('check', 'broken'), 165)
(('speed', 'handpiece'), 162)
(('bone', 'expander'), 158)
(('germany', 'insurance'), 157)
(('syst

(('bucksjobs', 'buckinghamshire'), 5)
(('snap', 'transfer'), 5)
(('system', 'coxo'), 5)
(('surgery', 'dentistry'), 5)
(('push', 'spray'), 5)
(('distances', 'check'), 5)
(('biohorizons', 'standard'), 5)
(('east', 'jobs'), 5)
(('platform', 'straight'), 5)
(('guide', 'check'), 5)
(('coverage', 'learn'), 5)
(('material', 'sterile'), 5)
(('surgical', 'micromotor'), 5)
(('angle', 'kavo'), 5)
(('type', 'azdent'), 5)
(('sizes', 'nobel'), 4)
(('workshop', 'yorbalinda'), 4)
(('surgical', 'ridge'), 4)
(('credit', 'card'), 4)
(('abutment', 'porcelain'), 4)
(('tricalcium', 'phosphate'), 4)
(('three', 'done'), 4)
(('lamina', 'straight'), 4)
(('abutment', 'attached'), 4)
(('analog', 'angulated'), 4)
(('yorbalinda', 'news'), 4)
(('standart', 'abutments'), 4)
(('inner', 'water'), 4)
(('payments', 'refunded'), 4)
(('lance', 'pilot'), 4)
(('micromotor', 'special'), 4)
(('done', 'surgery'), 4)
(('premium', 'full'), 4)
(('wrench', 'oral'), 4)
(('awareness', 'month'), 4)
(('osstem', 'dentium'), 4)
(('losing

(('motor', 'nordin'), 2)
(('jonathan', 'nash'), 2)
(('motor', 'coxo'), 2)
(('handpiece', 'svtn'), 2)
(('manager', 'basel'), 2)
(('analog', 'push'), 2)
(('tools', 'motor'), 2)
(('replacement', 'intra'), 2)
(('locator', 'gauge'), 2)
(('coxo', 'unit'), 2)
(('benefit', 'discover'), 2)
(('enter', 'biomet'), 2)
(('dentalimplants', 'medicalinsurance'), 2)
(('analog', 'quator'), 2)
(('extra', 'drills'), 2)
(('know', 'insurance'), 2)
(('tool', 'drivers'), 2)
(('motor', 'nouvag'), 2)
(('check', 'great'), 2)
(('angle', 'disease'), 2)
(('compression', 'prosthodontic'), 2)
(('free', 'teeth'), 2)
(('attachment', 'hard'), 2)
(('implantation', 'large'), 2)
(('check', 'trephine'), 2)
(('ebay', 'gift'), 2)
(('structure', 'screw'), 2)
(('iqox', 'push'), 2)
(('related', 'check'), 2)
(('mill', 'crusher'), 2)
(('fixation', 'titanium'), 2)
(('bone', 'mallet'), 2)
(('shopping', 'price'), 2)
(('treatment', 'call'), 2)
(('regional', 'sales'), 2)
(('consultation', 'financing'), 2)
(('ebay', 'sniping'), 2)
(('syd

(('spray', 'reduction'), 1)
(('dlls', 'accept'), 1)
(('page', 'biomet'), 1)
(('delivery', 'charges'), 1)
(('rogen', 'lose'), 1)
(('cases', 'idea'), 1)
(('socialbuttons', 'taking'), 1)
(('home', 'travel'), 1)
(('activities', 'calendar'), 1)
(('cover', 'reduction'), 1)
(('twitter', 'social'), 1)
(('hove', 'want'), 1)
(('weight', 'abutment'), 1)
(('azdent', 'biomet'), 1)
(('cost', 'warehouse'), 1)
(('please', 'share'), 1)
(('fairfax', 'vegas'), 1)
(('first', 'includes'), 1)
(('ohio', 'whether'), 1)
(('cell', 'idea'), 1)
(('using', 'dask'), 1)
(('loan', 'hardship'), 1)
(('crown', 'spokane'), 1)
(('kavo', 'sales'), 1)
(('view', 'idea'), 1)
(('veneers', 'tooth'), 1)
(('cica', 'check'), 1)
(('heel', 'sandals'), 1)
(('kaiser', 'redwoodcity'), 1)
(('dairy', 'creamer'), 1)
(('motor', 'ratio'), 1)
(('available', 'robot'), 1)
(('cost', 'special'), 1)
(('machine', 'state'), 1)
(('civic', 'phoenix'), 1)
(('cicada', 'ratio'), 1)
(('website', 'details'), 1)
(('paisley', 'jackets'), 1)
(('belton', 'mon

(('clothes', 'shopping'), 1)
(('certain', 'contemporary'), 1)
(('filling', 'crack'), 1)
(('marketing', 'covered'), 1)
(('solid', 'abutment'), 1)
(('light', 'health'), 1)
(('internal', 'standard'), 1)
(('paws', 'read'), 1)
(('instruments', 'tray'), 1)
(('pakistan', 'exhibition'), 1)
(('right', 'straight'), 1)
(('coping', 'healing'), 1)
(('finance', 'vicksburg'), 1)
(('surgery', 'styl'), 1)
(('type', 'please'), 1)
(('smile', 'info'), 1)
(('unbelievable', 'grand'), 1)
(('plans', 'sword'), 1)
(('dispensers', 'price'), 1)
(('plans', 'available'), 1)
(('today', 'news'), 1)
(('bestdentalmplants', 'placed'), 1)
(('environmental', 'specialist'), 1)
(('specialist', 'saudi'), 1)
(('fedi', 'periodontal'), 1)
(('poly', 'strapping'), 1)
(('teaching', 'matter'), 1)
(('individuals', 'neil'), 1)
(('straightening', 'spray'), 1)
(('idea', 'handpiece'), 1)
(('super', 'azdent'), 1)
(('carolina', 'arnold'), 1)
(('instruments', 'total'), 1)
(('automobile', 'insurance'), 1)
(('laserjet', 'portland'), 1)
(('sc

...

(('dentalimplant', 'surgery'), 1)
(('brush', 'reduction'), 1)
(('expat', 'health'), 1)
(('instrument', 'sterile'), 1)
(('researchers', 'columbia'), 1)
(('internal', 'deals'), 1)
(('clinical', 'laboratory'), 1)
(('orthodontist', 'dentists'), 1)
(('fresno', 'oral'), 1)
(('additivemanufacturing', 'failed'), 1)
(('trading', 'calculators'), 1)
(('whether', 'medical'), 1)
(('quote', 'medical'), 1)
(('drivers', 'drivers'), 1)
(('spray', 'hruy'), 1)
(('handpiece', 'cost'), 1)
(('tonsillitis', 'swollen'), 1)
(('tooth', 'sandent'), 1)
(('doctors', 'specialist'), 1)
(('highland', 'clinic'), 1)
(('clipart', 'leaves'), 1)
(('machine', 'sandent'), 1)
(('delivery', 'insurance'), 1)
(('offer', 'cannot'), 1)
(('implantesdentales', 'dentistlife'), 1)
(('angle', 'machine'), 1)
(('nearly', 'people'), 1)
(('business', 'card'), 1)
(('start', 'publishing'), 1)
(('lose', 'weight'), 1)
(('costs', 'year'), 1)
(('oral', 'surgical'), 1)
(('shopping', 'mini'), 1)
(('handpiece', 'unit'), 1)
(('forms', 'insurance'),

(('tools', 'year'), 1)
(('fractured', 'check'), 1)
(('cicada', 'youtube'), 1)
(('ship', 'bone'), 1)
(('pregnant', 'covered'), 1)
(('price', 'include'), 1)
(('dynamic', 'navigation'), 1)
(('inspire', 'phantom'), 1)
(('anatotemp', 'anatomic'), 1)
(('commercial', 'card'), 1)
(('subaru', 'insurance'), 1)
(('analog', 'dentures'), 1)
(('spring', 'columbia'), 1)
(('maxillofacial', 'surgeries'), 1)
(('gowelln', 'rent'), 1)
(('catering', 'diego'), 1)
(('abutment', 'hawaiifamilydental'), 1)
(('reynosa', 'reddit'), 1)
(('speed', 'braces'), 1)
(('sunsetsmilesrecommend', 'biomet'), 1)
(('lansingmi', 'stjohnsmi'), 1)
(('dentalimplants', 'fullmouthrehabilitation'), 1)
(('plans', 'mysterio'), 1)
(('available', 'material'), 1)
(('angled', 'shoulder'), 1)
(('specialist', 'reddit'), 1)
(('internal', 'coxo'), 1)
(('clearchoice', 'cosil'), 1)
(('belton', 'biomet'), 1)
(('expensive', 'pick'), 1)
(('iridium', 'communications'), 1)
(('coverage', 'roofer'), 1)
(('desks', 'sale'), 1)
(('reduction', 'contrangle'

(('suitcase', 'brushless'), 1)
(('listings', 'system'), 1)
(('realtors', 'insurance'), 1)
(('information', 'anatomic'), 1)
(('catalog', 'shopping'), 1)
(('idaho', 'falls'), 1)
(('dairy', 'lane'), 1)
(('cheap', 'performed'), 1)
(('medical', 'tooth'), 1)
(('plans', 'company'), 1)
(('insurance', 'mastercardsecurecode'), 1)
(('parallel', 'wall'), 1)
(('dentistry', 'fairfax'), 1)
(('royal', 'images'), 1)
(('grafting', 'instrument'), 1)
(('standard', 'anti'), 1)
(('overdose', 'affordable'), 1)
(('instruments', 'whatsapp'), 1)
(('assemble', 'home'), 1)
(('titanium', 'sponsored'), 1)
(('spray', 'motor'), 1)
(('medical', 'dentistry'), 1)
(('approach', 'sinus'), 1)
(('harmony', 'special'), 1)
(('ovarian', 'cancer'), 1)
(('impression', 'home'), 1)
(('plans', 'hernia'), 1)
(('locator', 'looking'), 1)
(('series', 'certified'), 1)
(('multiple', 'abutment'), 1)
(('screw', 'membrane'), 1)
(('pain', 'houston'), 1)
(('insurance', 'want'), 1)
(('london', 'salary'), 1)
(('tooth', 'cavity'), 1)
(('ankylos'

(('charleston', 'single'), 1)
(('tablet', 'online'), 1)
(('scrapers', 'curved'), 1)
(('football', 'gloves'), 1)
(('gloves', 'online'), 1)
(('seized', 'income'), 1)
(('chuck', 'check'), 1)
(('free', 'blades'), 1)
(('bone', 'preparation'), 1)
(('analytical', 'mechanics'), 1)
(('home', 'loan'), 1)
(('implantwarranty', 'extractions'), 1)
(('anatomic', 'comp'), 1)
(('insurance', 'hotels'), 1)
(('shopping', 'riyadh'), 1)
(('wireless', 'indoor'), 1)
(('texas', 'commercial'), 1)
(('abimplant', 'dentist'), 1)
(('literature', 'depression'), 1)
(('southerland', 'funeral'), 1)
(('hollywoodsmile', 'clinics'), 1)
(('insurance', 'kindle'), 1)
(('trilobe', 'reduction'), 1)
(('measurement', 'consultation'), 1)
(('available', 'current'), 1)
(('schedule', 'pipe'), 1)
(('tools', 'biohorizons'), 1)
(('quality', 'company'), 1)
(('motor', 'straumann'), 1)
(('periodontic', 'check'), 1)
(('insurance', 'paws'), 1)
(('says', 'angela'), 1)
(('stores', 'knoxville'), 1)
(('surgical', 'parallel'), 1)
(('lender', 'co

(('kong', 'insurance'), 1)
(('hair', 'smooth'), 1)
(('better', 'insurance'), 1)
(('covers', 'anything'), 1)
(('insurance', 'drivers'), 1)
(('ankylos', 'kits'), 1)
(('analog', 'conical'), 1)
(('cleaning', 'special'), 1)
(('handpiece', 'fsgi'), 1)
(('comfortable', 'call'), 1)
(('single', 'aesthetic'), 1)
(('medium', 'roast'), 1)
(('sale', 'dentist'), 1)
(('riverchase', 'check'), 1)
(('kimberly', 'clark'), 1)
(('benefits', 'smile'), 1)
(('already', 'broken'), 1)
(('dentium', 'idea'), 1)
(('specialist', 'cheap'), 1)
(('shape', 'enables'), 1)
(('successfully', 'highly'), 1)
(('compensation', 'insurance'), 1)
(('catalog', 'caturday'), 1)
(('time', 'dentalopolis'), 1)
(('welcomes', 'practice'), 1)
(('auctions', 'rare'), 1)
(('cost', 'literature'), 1)
(('wrench', 'elite'), 1)
(('georgehas', 'found'), 1)
(('torque', 'wrench'), 1)
(('best', 'affordable'), 1)
(('machine', 'washing'), 1)
(('creamer', 'fattening'), 1)
(('details', 'anxiety'), 1)
(('dentistry', 'kits'), 1)
(('bill', 'broken'), 1)
((

In [19]:
tgs = ngrams(clean_tokens,3)
aaa = Counter(tgs).most_common()
for b in aaa:
    print(b)

(('cost', 'surprise', 'sponsored'), 849)
(('idea', 'cost', 'surprise'), 849)
(('surprise', 'sponsored', 'listings'), 849)
(('sponsored', 'listings', 'idea'), 579)
(('listings', 'idea', 'cost'), 579)
(('contra', 'angle', 'handpiece'), 518)
(('save', 'money', 'costa'), 414)
(('money', 'costa', 'rica'), 414)
(('costa', 'rica', 'save'), 407)
(('rica', 'save', 'money'), 407)
(('speed', 'contra', 'angle'), 383)
(('reduction', 'speed', 'contra'), 320)
(('restoration', 'surgery', 'tools'), 305)
(('dentistry', 'restoration', 'surgery'), 305)
(('reduction', 'contra', 'angle'), 203)
(('surgical', 'straight', 'handpiece'), 173)
(('straight', 'handpiece', 'nose'), 171)
(('insurance', 'bowling', 'lanes'), 167)
(('bowling', 'lanes', 'germany'), 167)
(('germany', 'insurance', 'bowling'), 157)
(('lanes', 'germany', 'insurance'), 157)
(('tools', 'abutment', 'analog'), 148)
(('surgery', 'tools', 'abutment'), 148)
(('check', 'broken', 'screw'), 147)
(('surgical', 'brushless', 'motor'), 138)
(('handpiece',

(('complete', 'range', 'instruments'), 4)
(('angle', 'handpiece', 'save'), 4)
(('inner', 'water', 'contra'), 4)
(('sets', 'system', 'screen'), 4)
(('human', 'futurist', 'abutment'), 4)
(('surgical', 'dentistry', 'restoration'), 4)
(('insurance', 'california', 'cost'), 4)
(('universal', 'drills', 'surgical'), 4)
(('micromotor', 'special', 'price'), 4)
(('drivers', 'universal', 'zahnimplantate'), 4)
(('listings', 'insurance', 'bowling'), 4)
(('alpha', 'biohorizons', 'dentistry'), 4)
(('slow', 'speed', 'contra'), 4)
(('surgical', 'surgery', 'instruments'), 4)
(('motor', 'reduction', 'contra'), 4)
(('drill', 'include', 'stoppers'), 4)
(('kavo', 'motors', 'ebaydailybids'), 4)
(('optic', 'handpiece', 'reduction'), 4)
(('prosthetics', 'access', 'prosthetics'), 4)
(('transfer', 'thread', 'check'), 4)
(('lamina', 'straight', 'abutment'), 4)
(('california', 'york', 'jersey'), 4)
(('bids', 'hiossen', 'esset'), 4)
(('threadform', 'biohorizons', 'external'), 4)
(('prosthetics', 'ebook', 'download')

(('medical', 'insurance', 'necessary'), 2)
(('home', 'early', 'seinfeld'), 2)
(('many', 'teeth', 'complete'), 2)
(('newimplantresearc', 'download', 'free'), 2)
(('listings', 'check', 'drill'), 2)
(('endo', 'surgical', 'motor'), 2)
(('whatsapp', 'office', 'idea'), 2)
(('endodontics', 'endo', 'motor'), 2)
(('button', 'handpiece', 'check'), 2)
(('cortex', 'zimmer', 'thommen'), 2)
(('complete', 'prosthetic', 'dentistry'), 2)
(('online', 'shopping', 'price'), 2)
(('drills', 'integral', 'stoppers'), 2)
(('prosthetics', 'read', 'free'), 2)
(('inclusive', 'includes', 'abutment'), 2)
(('cicada', 'contra', 'angle'), 2)
(('sales', 'specialist', 'وظائف'), 2)
(('user', 'check', 'surgery'), 2)
(('check', 'insurance', 'covers'), 2)
(('processing', 'check', 'insurance'), 2)
(('handpiece', 'disease', 'teeth'), 2)
(('bridge', 'better', 'approach'), 2)
(('made', 'titanium', 'abutment'), 2)
(('whatsapp', 'reduction', 'speed'), 2)
(('angle', 'handpiece', 'fiber'), 2)
(('sale', 'motor', 'system'), 2)
(('isr

(('abutment', 'crown', 'today'), 2)
(('angle', 'handpiece', 'learn'), 2)
(('style', 'azdent', 'reduction'), 2)
(('metal', 'allergy', 'check'), 2)
(('demornay', 'episode', 'even'), 2)
(('bego', 'check', 'screw'), 2)
(('surgical', 'repair', 'fast'), 2)
(('insurance', 'covered', 'abutment'), 2)
(('free', 'teeth', 'whitening'), 2)
(('prosthetics', 'ebook', 'free'), 2)
(('analog', 'check', 'fractured'), 2)
(('digitally', 'planned', 'abutment'), 2)
(('double', 'spray', 'handpiece'), 2)
(('prosthetic', 'dentistry', 'restoration'), 2)
(('quality', 'tool', 'drivers'), 2)
(('phoenix', 'sales', 'consultant'), 2)
(('help', 'dentistry', 'restoration'), 2)
(('analog', 'lock', 'abutment'), 2)
(('abutment', 'analog', 'push'), 2)
(('handpiece', 'speed', 'check'), 2)
(('handpiece', 'sinus', 'lift'), 2)
(('ship', 'reduction', 'contra'), 2)
(('without', 'insurance', 'news'), 2)
(('push', 'button', 'jinme'), 2)
(('usps', 'sale', 'stock'), 2)
(('meter', 'wireless', 'mini'), 2)
(('machine', 'drill', 'motor')

(('coverage', 'online', 'shop'), 2)
(('handpiece', 'super', 'mini'), 2)
(('help', 'chinese', 'robot'), 2)
(('loss', 'call', 'schedule'), 2)
(('drill', 'complete', 'handpiece'), 2)
(('button', 'handpiece', 'dentistry'), 2)
(('conical', 'connection', 'straight'), 2)
(('implantation', 'implantat', 'surgery'), 2)
(('inevitable', 'instruments', 'implantation'), 2)
(('ruler', 'zahnimplantate', 'check'), 2)
(('retained', 'cemented', 'abutments'), 2)
(('angle', 'handpiece', 'enter'), 2)
(('azdent', 'handpiece', 'surgical'), 2)
(('full', 'surgical', 'surgical'), 2)
(('speed', 'robot', 'successfully'), 2)
(('abutment', 'analog', 'good'), 2)
(('kindle', 'free', 'complications'), 2)
(('monthly', 'payments', 'talk'), 2)
(('futurist', 'system', 'micromotor'), 2)
(('sale', 'check', 'bone'), 2)
(('stakepoolcom', 'robot', 'completes'), 2)
(('free', 'straight', 'abutment'), 2)
(('check', 'sandent', 'reduction'), 2)
(('iframe', 'width', 'height'), 2)
(('oregon', 'portland', 'affordable'), 2)
(('holds', '

...

(('instruments', 'instruments', 'lowest'), 1)
(('altogether', 'bioconcept', 'persistent'), 1)
(('current', 'trends', 'beginnings'), 1)
(('offers', 'abutment', 'dentalimplants'), 1)
(('implantdirect', 'bego', 'interviewing'), 1)
(('silverline', 'pakistan', 'exhibition'), 1)
(('comment', 'like', 'free'), 1)
(('right', 'surgical', 'contra'), 1)
(('complete', 'zimmer', 'dublin'), 1)
(('handpiece', 'suitcase', 'initial'), 1)
(('consultation', 'including', 'dentalimplants'), 1)
(('caps', 'internal', 'idea'), 1)
(('companies', 'cover', 'cost'), 1)
(('premium', 'full', 'basic'), 1)
(('free', 'science', 'advertising'), 1)
(('whatsapp', 'office', 'hiossen'), 1)
(('release', 'tooth', 'sydney'), 1)
(('contra', 'angle', 'affordable'), 1)
(('brushless', 'motor', 'smile'), 1)
(('free', 'consultation', 'insurance'), 1)
(('surgical', 'idea', 'cost'), 1)
(('title', 'mini', 'michigan'), 1)
(('insurance', 'cover', 'whole'), 1)
(('inverter', 'online', 'shopping'), 1)
(('zimmer', 'light', 'reduction'), 1)
(

(('sausage', 'last', 'insurance'), 1)
(('angle', 'handpiece', 'actually'), 1)
(('insurance', 'cover', 'video'), 1)
(('steamboat', 'springs', 'analysis'), 1)
(('optimized', 'orientation', 'allows'), 1)
(('free', 'insurance', 'apple'), 1)
(('drivers', 'megagen', 'motor'), 1)
(('call', 'free', 'minute'), 1)
(('london', 'free', 'cheap'), 1)
(('unbelievable', 'grand', 'opening'), 1)
(('plan', 'insurance', 'specialist'), 1)
(('clinical', 'laboratory', 'manual'), 1)
(('plans', 'farm', 'liability'), 1)
(('generally', 'covered', 'need'), 1)
(('bumper', 'bumper', 'warranty'), 1)
(('coffee', 'morning', 'saturday'), 1)
(('worth', 'texas', 'tutoring'), 1)
(('dentists', 'tired', 'fees'), 1)
(('affordable', 'sports', 'cars'), 1)
(('sale', 'machine', 'system'), 1)
(('damagetooth', 'great', 'insurance'), 1)
(('prices', 'insurance', 'refuse'), 1)
(('food', 'memphis', 'housing'), 1)
(('gettu', 'idea', 'cost'), 1)
(('primetime', 'schedule', 'swot'), 1)
(('free', 'exam', 'patients'), 1)
(('covered', 'insur

(('dentalimplantinstruments', 'pitchatpalace', 'africa'), 1)
(('tiffany', 'light', 'best'), 1)
(('implantologia', 'super', 'interesting'), 1)
(('save', 'money', 'ways'), 1)
(('insurance', 'laws', 'scottsdale'), 1)
(('pricey', 'often', 'best'), 1)
(('preview', 'outcomes', 'immediate'), 1)
(('flip', 'cover', 'online'), 1)
(('screw', 'screw', 'removal'), 1)
(('prosthetics', 'portland', 'portlandhow'), 1)
(('medical', 'dentistry', 'restoration'), 1)
(('enter', 'system', 'brushless'), 1)
(('rare', 'system', 'drill'), 1)
(('india', 'sales', 'portland'), 1)
(('portland', 'oregon', 'attorneys'), 1)
(('benefits', 'insurance', 'abutment'), 1)
(('straumann', 'biomet', 'totalimplant'), 1)
(('medical', 'industries', 'lansdale'), 1)
(('instruments', 'kits', 'crown'), 1)
(('traus', 'motor', 'machine'), 1)
(('individuals', 'travelocity', 'flight'), 1)
(('give', 'call', 'super'), 1)
(('home', 'furnishing', 'importers'), 1)
(('prosthetic', 'tooth', 'pembroke'), 1)
(('include', 'tooth', 'colored'), 1)
((

(('speed', 'reduction', 'handpiece'), 1)
(('details', 'advertising', 'bingads'), 1)
(('like', 'income', 'insurance'), 1)
(('angle', 'ebaydailybids', 'speed'), 1)
(('short', 'titanium', 'sleeve'), 1)
(('lift', 'tools', 'periosteal'), 1)
(('texarkana', 'gazette', 'jawboneup'), 1)
(('expander', 'disk', 'idea'), 1)
(('individuals', 'truck', 'sale'), 1)
(('catalog', 'fort', 'myers'), 1)
(('club', 'insurance', 'company'), 1)
(('wrench', 'tenax', 'check'), 1)
(('speed', 'affordable', 'insurance'), 1)
(('covers', 'cost', 'insurance'), 1)
(('thread', 'abutment', 'types'), 1)
(('analog', 'replace', 'single'), 1)
(('repost', 'smileplushomestead', 'consider'), 1)
(('elevators', 'periosteal', 'dentistry'), 1)
(('surgical', 'tools', 'businesslisting'), 1)
(('post', 'surgical', 'style'), 1)
(('health', 'insurance', 'organization'), 1)
(('coverage', 'mazda', 'mile'), 1)
(('angle', 'reduction', 'surgery'), 1)
(('money', 'prosthetics', 'save'), 1)
(('tooth', 'advdentalartsnj', 'somersetnj'), 1)
(('parsi

(('page', 'affordable', 'asset'), 1)
(('find', 'offer', 'ends'), 1)
(('irrigation', 'surgical', 'free'), 1)
(('online', 'shop', 'anti'), 1)
(('abutment', 'bioconcept', 'package'), 1)
(('science', 'surgical', 'prep'), 1)
(('greensky', 'specialists', 'save'), 1)
(('cover', 'insurance', 'health'), 1)
(('platform', 'mini', 'head'), 1)
(('surgical', 'assistant', 'seeking'), 1)
(('price', 'involved', 'losing'), 1)
(('kalonji', 'online', 'shopping'), 1)
(('certain', 'prevail', 'push'), 1)
(('music', 'gettu', 'idea'), 1)
(('makeover', 'enter', 'trephine'), 1)
(('spray', 'handpiece', 'instrument'), 1)
(('degree', 'free', 'movement'), 1)
(('locator', 'attachment', 'landlord'), 1)
(('mini', 'ideas', 'transition'), 1)
(('cream', 'insurance', 'project'), 1)
(('implantspecialist', 'abutment', 'analog'), 1)
(('management', 'software', 'much'), 1)
(('mallet', 'well', 'download'), 1)
(('combo', 'insurance', 'covers'), 1)
(('insurance', 'covers', 'toner'), 1)
(('dentist', 'biomet', 'full'), 1)
(('tire',